# Convert a directory of CSV files downloaded from Veloview to pcd files

In [ ]:
!pip install pypcd

  Created wheel for python-lzf: filename=python_lzf-0.2.4-cp37-cp37m-linux_x86_64.whl size=23027 sha256=5d1d2e509636d4a96cd6846dba0187259e640482d5f9aafec4f73430e4d0ebed
  Stored in directory: /root/.cache/pip/wheels/d1/5b/05/473bf0cd62eae63099399436edbaa0d54ae8de76c130d6d970
Successfully built python-lzf


In [ ]:
import getopt,os,sys,csv,numpy as np
from io import StringIO
from pypcd import pypcd
import pprint
import pandas as pd  
import os
import glob

In [ ]:
# use glob to get all the csv files 
# in the folder
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

In [ ]:
csv_files.sort()

In [ ]:
from tqdm import tqdm
name = 0
for f in tqdm(range(csv_files)):  
    
    # read the csv file
    data = pd.read_csv(csv_files[f])
    cloud = np.array(data)
    #print(cloud.shape)
    # take X,Y,Z coordinates
    X = cloud[:, 7]
    Y = cloud[:, 8]
    Z = cloud[:, 9]

    X = X.astype(np.float32).reshape(1, -1)
    Y = Y.astype(np.float32).reshape(1, -1)
    Z = Z.astype(np.float32).reshape(1, -2) 

    blue = (Y/np.max(Y))
    n = Y.size
    rgb = np.hstack((np.repeat(0.01, n)[:, np.newaxis], np.repeat(0.01, n)[:, np.newaxis], blue.T))
    encoded_colors = pypcd.encode_rgb_for_pcl((rgb * 255).astype(np.uint8))
    new_data = np.hstack((X.T, Y.T, Z.T, encoded_colors[:, np.newaxis]))

    len = new_data.size

    md = {'version': .7,
          'fields': ['x', 'y', 'z'],
          'size': [4, 4, 4],
          'type': ['F', 'F', 'F'],
          'count': [1, 1, 1],
          'width': len,
          'height': 1,
          'viewpoint': [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
          'points': len,
          'data': 'ascii'}

    new_cloud = pypcd.make_xyz_rgb_point_cloud(new_data)
    new_cloud.save( str(name).zfill(4) +'.pcd')
    name = name + 1


100%|██████████| 100/100 [01:44<00:00,  1.05s/it]
